In [ ]:
import numpy as np 
import cv2 
import matplotlib.pyplot as plt 
from scipy.interpolate import NearestNDInterpolator
from math import * 
plt.rcParams['figure.figsize'] = [16, 12]
from scipy.spatial import Delaunay


In [ ]:
img_1 = cv2.imread("./inputs/P4/kobytev_1941.png",0)[:,:771]
img_2 = cv2.imread("./inputs/P4/kobytev_1945.png",0)


### #a

In [ ]:
plt.rcParams['figure.figsize'] = [16, 12]
def NaiveMethod(img_1,img_2):
    i = 1
    for index in np.arange(0,1.05,0.05):
        plt.subplot(4,5,i)
        new_img = img_1*(1-index) + img_2*index
        plt.imshow(new_img,cmap="gray")
        plt.axis("off")
        plt.title("Naive method with a="+str(round(index,2)))
        i+=1
        if i == 21 :
            break
        
    plt.tight_layout()
    plt.show()
    
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    h,w = img_1.shape
    out = cv2.VideoWriter("./outputs/P04/a-output.mp4", fourcc, 30, (w, h))
    
    for a in np.arange(0,1,1/60):
        new_img = img_1*(1-a) + img_2*a 
        new_img = cv2.cvtColor(new_img.astype(np.uint8),cv2.COLOR_GRAY2RGB)
        out.write(new_img)
    out.release()
   
NaiveMethod(img_1,img_2)  

### #b

In [ ]:
plt.rcParams['figure.figsize'] = [8, 6]
from datetime import datetime


def event_handler(event, p_x, p_y, _ , param):
    if event == cv2.EVENT_LBUTTONDOWN:
        param[1].append([p_x, p_y])
        cv2.destroyWindow("GetKeypoints")
        cv2.circle(param[0], param[1][-1], 5, (242, 80, 91), -1)            
        point_picker(param[0],param[1],param[2])
    else:
        pass

def point_picker(img, selected_points,count=26):
    if len(img.shape) != 3 :
        img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        
    if len(selected_points) == count :
        plt.imshow(img)
        plt.axis("off")
        plt.title("Selected KeyPoints")
        plt.show()
        return selected_points    
    cv2.imshow('GetKeypoints', img[:,:,::-1])
    cv2.setMouseCallback('GetKeypoints', event_handler, [img, selected_points,count])
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return selected_points


points_1 = np.array(point_picker(img_1.copy(),[]))
points_2 = np.array(point_picker(img_2.copy(),[]))

with open('./outputs/P04/b-points-1-2.npy', 'wb') as f:
    np.save(f, points_1)

with open('./outputs/P04/b-points-2-2.npy', 'wb') as f:
    np.save(f, points_2)

In [ ]:
with open('./outputs/P04/b-points-1.npy', 'rb') as f:
    points_1 = np.load(f)

with open('./outputs/P04/b-points-2.npy', 'rb') as f:
    points_2 = np.load(f)
points_2[21][0] -= 20  



In [ ]:
plt.rcParams['figure.figsize'] = [12, 9]

imgs = [ img_1, img_2]
points = [points_1,points_2]
d = [1941,1945 ]
for i in range(2):
    img = cv2.cvtColor(imgs[i],cv2.COLOR_GRAY2RGB)
    _points = points[i]
    plt.subplot(1,2,i+1)
    index = 1
    for p in _points : 
        t = p.copy()
        cv2.circle(img, t, 5, (242, 80, 91), -1)   
        cv2.putText(img=img, text=str(index), org=t+5, fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=0.75, color=(242, 80, 91),thickness=2)
        index +=1
    plt.imshow(img)
    plt.axis("off")
    plt.title("Selected keypoints for image {0}".format(d[i]))

plt.tight_layout()
plt.show()

### #c

In [ ]:
cornerss_1 = [ 
            [0,0],
            [0,img_1.shape[0]-1],
            [img_1.shape[1]-1,0],
            [img_1.shape[1]-1,img_1.shape[0]-1]
            ]

cornerss_2 = [ 
            [0,0],
            [0,img_2.shape[0]-1],
            [img_2.shape[1]-1,0],
            [img_2.shape[1]-1,img_2.shape[0]-1]
            ]
points_1_full = np.append(points_1,cornerss_1,axis=0)
points_2_full = np.append(points_2,cornerss_2,axis=0)
fulls = [points_1_full,points_2_full]
imgs = [ img_1, img_2]
points = [points_1,points_2]
cors = [cornerss_1,cornerss_2]
d = [1941,1945 ]

for i in range(2):
    img = cv2.cvtColor(imgs[i],cv2.COLOR_GRAY2RGB)
    _f = fulls[i]
    
    plt.subplot(1,2,i+1)
    plt.imshow(img)
    
    tri = Delaunay(_f)
    plt.triplot(_f[:,0], _f[:,1], tri.simplices,color="blue",alpha=0.35)
    plt.plot(_f[:,0], _f[:,1], '*',color="red")

    plt.axis("off")
    plt.title("Triangulation for image {0}".format(d[i]))

plt.tight_layout()
plt.show()

### #d

In [ ]:
plt.rcParams['figure.figsize'] = [16, 12]

def get_position_by_indice(tri_1,position):
    n= tri_1.shape[0]
    tri_1 = np.reshape(tri_1,(-1,))
    li = []
    for t in tri_1:
       li.append(position[t]) 
       
    res = np.reshape(li,(n,3,2))
    return res 

def get_single_frame(scr_ps,dst_p,img):
    Rs = []
    for rig_i in range(scr_ps.shape[0]):
        mask = np.zeros_like(img,dtype=np.uint8)
        cv2.fillPoly(mask, pts=[scr_ps[rig_i]], color=(255, 255, 255))
        warp_mat_1 = cv2.getAffineTransform(scr_ps[rig_i].astype(np.float32), dst_p[rig_i].astype(np.float32))
        warp_dst_1 = cv2.warpAffine(img, warp_mat_1, (img.shape[1], img.shape[0]))
        warp_dst_1_mask = cv2.warpAffine(mask, warp_mat_1, (img.shape[1], img.shape[0]))
        
        result = cv2.bitwise_and(warp_dst_1, warp_dst_1, mask=warp_dst_1_mask)
        Rs.append(result)
     
    return sum(Rs)

def get_single_image(a,in1,in2):
    dst_ps = (1-a)*in1 + a*in2
    p1 = get_single_frame(in1,dst_ps,img_1.copy())
    p2 = get_single_frame(in2,dst_ps,img_2.copy())
    p = (1-a)*p1 + a*p2
    
    return p
    
tri_1 = Delaunay(points_1_full).simplices

def handle():
    in1 = get_position_by_indice(tri_1,fulls[0])
    in2 = get_position_by_indice(tri_1,fulls[1])
    
    index = 1
    for alpha in np.arange(0,1,1/10):
        plt.subplot(2,5,index)
        index += 1
        image = get_single_image(alpha,in1,in2)
        plt.imshow(image,cmap="gray")
        plt.axis("off")
        plt.title("Triangulation method with a="+str(round(alpha,2)))
        
    plt.tight_layout()
    plt.show()
    
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    h,w = img_1.shape
    out = cv2.VideoWriter("./outputs/P04/d-output.mp4", fourcc, 30, (w, h))
    
    for a in np.arange(0,1,1/60):
        new_img = get_single_image(a,in1,in2)
        new_img = cv2.cvtColor(new_img.astype(np.uint8),cv2.COLOR_GRAY2RGB)
        out.write(new_img)
        
        
    new_img = get_single_image(1,in1,in2)
    new_img = cv2.cvtColor(new_img.astype(np.uint8),cv2.COLOR_GRAY2RGB)
    out.write(new_img)
    out.release()
    
    
handle()

In [ ]:
in1 = get_position_by_indice(tri_1,fulls[0])
in2 = get_position_by_indice(tri_1,fulls[1])

index = 1
for alpha in np.arange(0,1,1/5):
    plt.subplot(4,5,index)
    index += 1
    image = get_single_image(alpha,in1,in2).astype("uint8")
    image = cv2.medianBlur(image,7)
    plt.imshow(image,cmap="gray")
    
plt.show()

In [ ]:
image